Library for Boxplot simility between Alphacode and Human code


In [1]:
import numpy as np
import os
import researchpy
import pandas as pd

Defined list of problem and contest index of each problem

In [2]:
listOfProblem= ['WebofLies','GregorandCryptography','PolycarpandCoins','WonderfulColoring','SubsequencePermutation','DigitsSum','RunningforGold',
'XORandDistance','Cherry','Cobb','Mikasa','AVarietyofOperations','EzzatandTwoSubsequences','MochaandMath','DislikeofThrees',
'SimplyStrangeSort','MedianMaximization','SeatingArrangementseasyversion','NonDecreasingDilemma','BalancedSubstring','Book',
'RegularBracketSequences']

nameMapProblem= {'WebofLies': '1549C','TheThreeLittlePigs': '1548C','GregorandtheTwoPainters': '1548E',
 'GregorandCryptography': '1549A','PolycarpandCoins': '1551A','WonderfulColoring': '1551B1',
 'SubsequencePermutation': '1552A','RunningforGold': '1552B','Backspace': '1553D','PermutationShift': '1553E',
 'DigitsSum': '1553A','XORandDistance': '1553H','Cherry': '1554A','Cobb': '1554B',
 'Mikasa': '1554C','SayNotoPalindromes': '1555D','AVarietyofOperations': '1556A','Moamenandksubarrays': '1557B',
 'EzzatandTwoSubsequences': '1557A','MochaandMath': '1559A','MochaandRedandBlue': '1559B','MochaandDianaEasyVersion': '1559D1',
 'DislikeofThrees': '1560A','InfinityTable': '1560C','NearestBeautifulNumbereasyversion': '1560F1','NearestBeautifulNumberhardversion': '1560F2',
 'SimplyStrangeSort': '1561A','ScenesFromaMemory': '1562B','SeatingArrangementseasyversion': '1566D1',
 'MINMEXCut': '1566B','MedianMaximization': '1566A','BudsRehanging': '1566E','NonDecreasingDilemma': '1567E',
 'BalancedSubstring': '1569A','Book': '1573C','Countdown': '1573A','RegularBracketSequences': '1574A',
 'CombinatoricsHomework': '1574B','MinimizeDistance': '1585C','MissingBigram': '1618B',
 "Singers'Tour": '1618E',"NewYear'sProblem": '1619D','GameonRanges': '1623B'}

This function for read file and normalize result file from Mprof textfile to list.

In [3]:
def getData(Path):
    mem,time = [],[]
    try:
        with open(f"{Path}") as f:
            for j in f.readlines()[1:]:
                tmp = j.split(' ')
                mem.append(float(tmp[1]))
                time.append(float(tmp[2][:-2]))
        return max(mem),time[-1]-time[0]
    except:
        return 0,0


This function for find Mem Mean,Median and SD peformance of Time and Mem usage between Alphacode and The most similar human code.

In [4]:
def getPerf(mainPath,lang,pl):
    res,resList={},{}
    for typ in ['alpha','sim']:
        timePerf,memPerf = [],[]
        dataPath = f"{mainPath}{lang.upper()}/{pl}/{lang}Perf/1/{typ}/{typ}"
        print(dataPath)
        for i in range(1,101):
            maxMem,maxTime = getData(f"{dataPath}{i}.txt")
            timePerf.append(maxTime)
            memPerf.append(maxMem)
        timepd = pd.Series(timePerf)
        mempd = pd.Series(memPerf)
        resList[typ] = {'time':timepd,'mem':mempd}
        timearr = np.array(timePerf)
        memarr = np.array(memPerf)
        res[typ]={'time':{'sd':round(np.std(timearr),4),'median':round(np.median(timearr),4),'mean':round(np.mean(timearr),4)} \
                ,'mem':{'sd':round(np.std(memarr),4),'median':round(np.median(memarr),4),'mean':round(np.mean(memarr),4)}}
    try:
        tmp1 = researchpy.ttest(resList['alpha']['time'],resList['sim']['time'],group1_name='AlphaCode',group2_name='Sim')
        tmp2 = researchpy.ttest(resList['alpha']['mem'],resList['sim']['mem'],group1_name='AlphaCode',group2_name='Sim')
    except:
        tmp1 = tmp2 = res = [0 for x in range(10)]
    return tmp1[1],tmp2[1],res

This function for interpret_cohen_d

In [5]:
def interpret_cohen_d(d_value):
    if 0 <= d_value < 0.1:
        return 'Very Small'
    elif 0.1 <= d_value < 0.35:
        return 'Small'
    elif 0.35 <= d_value < 0.65:
        return 'Medium'
    elif 0.65 <= d_value < 0.9:
        return 'Large'
    elif d_value >= 0.9:
        return 'Very Large'
    else:
        return 'Unknown'

This function to find stistical test and save table to csv.

In [6]:
def Write_All_Stistic_csv(listOfProblem,lang,res2lang,lperfmem,lperftime):
    resTimeDF = pd.DataFrame()
    resMemDF = pd.DataFrame()
    for i in listOfProblem:
        tmp = []
        for j in ['alpha','sim']:
            for k in ['time','mem']:
                try:
                    mean = res2lang[i][j][k]['mean']
                    med = res2lang[i][j][k]['median']
                    sd = res2lang[i][j][k]['sd']
                except:
                    mean =0
                    med=0
                    sd = 0
                tmp.append((mean,med,sd))
        check = lambda x : x < 0.0001
        if(nameMapProblem[i] not in ['1553H','1554B','1567E']):
            resTimeDF = resTimeDF.append({'name':nameMapProblem[i],'alTimeMean':tmp[0][0],'alTimeMed':tmp[0][1],'alTimeSD':tmp[0][2], \
                                        'simTimeMean':tmp[2][0],'simTimeMed':tmp[2][1],'simTimeSD':tmp[2][2], \
                                        'delta':abs(round(tmp[0][1]-tmp[2][1],4)), \
                                        't-test(p-value < 0.001)':f"{lperftime[i]['results'][3]}, {check(lperftime[i]['results'][3])}","Cohen's d":f"{abs(round(lperftime[i]['results'][6],2))} ({interpret_cohen_d(abs(lperftime[i]['results'][6]))})"},\
                                        ignore_index=True)
            resMemDF = resMemDF.append({'name':nameMapProblem[i],'alMemMean':tmp[1][0],'alMemMed':tmp[1][1],'alMemSD':tmp[1][2], \
                                        'simMemMean':tmp[3][0],'simMemMed':tmp[3][1],'simMemSD':tmp[3][2], \
                                        'delta':abs(round(tmp[1][1]-tmp[3][1],4)), \
                                    't-test(p-value < 0.001)':f"{lperfmem[i]['results'][3]}, {check(lperfmem[i]['results'][3])}","Cohen's d":f"{abs(round(lperfmem[i]['results'][6],2))} ({interpret_cohen_d(abs(lperfmem[i]['results'][6]))})"},\
                                    ignore_index=True)
        else:
            resTimeDF = resTimeDF.append({'name':nameMapProblem[i],'alTimeMean':tmp[0][0],'alTimeMed':tmp[0][1],'alTimeSD':tmp[0][2], \
                                        'simTimeMean':'NA','simTimeMed':'NA','simTimeSD':'NA', \
                                        'delta':'NA', \
                                        't-test(p-value < 0.001)':'NA',"Cohen's d":"NA"},\
                                        ignore_index=True)
            resMemDF = resMemDF.append({'name':nameMapProblem[i],'alMemMean':tmp[0][0],'alMemMed':tmp[0][1],'alMemSD':tmp[0][2], \
                                        'simMemMean':'NA','simMemMed':'NA','simMemSD':'NA', \
                                        'delta':'NA', \
                                        't-test(p-value < 0.001)':'NA',"Cohen's d":"NA"},\
                                        ignore_index=True)
    print(resMemDF.sort_values(by='name'))
    
    resMemDF.sort_values(by='name').to_csv(f'{lang}Mem.csv',index=False)
    print(resTimeDF.sort_values(by='name'))
    resTimeDF.sort_values(by='name').to_csv(f'{lang}time.csv',index=False)





Function call for all problem in both 2 languages.

In [7]:
mainPath='/Users/silalertbanjongngam/Desktop/CodeInTheResearch/RawData/'
for l in ['Cpp','Python']:
    res2lang={}    
    lperfmem={}
    lperftime={}
    print('Language: ', l)
    for i in listOfProblem:
        print('name: ',i)
        lperftime[i],lperfmem[i],res2lang[i] = getPerf(mainPath,l,i)
    Write_All_Stistic_csv(listOfProblem,l,res2lang,lperfmem,lperftime)

Language:  Cpp
name:  WebofLies
/Users/silalertbanjongngam/Desktop/CodeInTheResearch/RawData/CPP/WebofLies/CppPerf/1/alpha/alpha
/Users/silalertbanjongngam/Desktop/CodeInTheResearch/RawData/CPP/WebofLies/CppPerf/1/sim/sim
name:  GregorandCryptography
/Users/silalertbanjongngam/Desktop/CodeInTheResearch/RawData/CPP/GregorandCryptography/CppPerf/1/alpha/alpha
/Users/silalertbanjongngam/Desktop/CodeInTheResearch/RawData/CPP/GregorandCryptography/CppPerf/1/sim/sim
name:  PolycarpandCoins
/Users/silalertbanjongngam/Desktop/CodeInTheResearch/RawData/CPP/PolycarpandCoins/CppPerf/1/alpha/alpha


/Users/silalertbanjongngam/anaconda3/lib/python3.9/site-packages/researchpy/ttest.py:18: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  groups = group1.append(group2, ignore_index= True)
/Users/silalertbanjongngam/anaconda3/lib/python3.9/site-packages/researchpy/ttest.py:18: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  groups = group1.append(group2, ignore_index= True)
/Users/silalertbanjongngam/anaconda3/lib/python3.9/site-packages/researchpy/ttest.py:18: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  groups = group1.append(group2, ignore_index= True)
/Users/silalertbanjongngam/anaconda3/lib/python3.9/site-packages/researchpy/ttest.py:18: FutureWarning: The series.append method is deprecated and will be removed from pand

/Users/silalertbanjongngam/Desktop/CodeInTheResearch/RawData/CPP/PolycarpandCoins/CppPerf/1/sim/sim
name:  WonderfulColoring
/Users/silalertbanjongngam/Desktop/CodeInTheResearch/RawData/CPP/WonderfulColoring/CppPerf/1/alpha/alpha
/Users/silalertbanjongngam/Desktop/CodeInTheResearch/RawData/CPP/WonderfulColoring/CppPerf/1/sim/sim
name:  SubsequencePermutation
/Users/silalertbanjongngam/Desktop/CodeInTheResearch/RawData/CPP/SubsequencePermutation/CppPerf/1/alpha/alpha
/Users/silalertbanjongngam/Desktop/CodeInTheResearch/RawData/CPP/SubsequencePermutation/CppPerf/1/sim/sim
name:  DigitsSum
/Users/silalertbanjongngam/Desktop/CodeInTheResearch/RawData/CPP/DigitsSum/CppPerf/1/alpha/alpha
/Users/silalertbanjongngam/Desktop/CodeInTheResearch/RawData/CPP/DigitsSum/CppPerf/1/sim/sim
name:  RunningforGold
/Users/silalertbanjongngam/Desktop/CodeInTheResearch/RawData/CPP/RunningforGold/CppPerf/1/alpha/alpha


/Users/silalertbanjongngam/anaconda3/lib/python3.9/site-packages/researchpy/ttest.py:18: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  groups = group1.append(group2, ignore_index= True)
/Users/silalertbanjongngam/anaconda3/lib/python3.9/site-packages/researchpy/ttest.py:18: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  groups = group1.append(group2, ignore_index= True)
/Users/silalertbanjongngam/anaconda3/lib/python3.9/site-packages/researchpy/ttest.py:18: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  groups = group1.append(group2, ignore_index= True)
/Users/silalertbanjongngam/anaconda3/lib/python3.9/site-packages/researchpy/ttest.py:18: FutureWarning: The series.append method is deprecated and will be removed from pand

/Users/silalertbanjongngam/Desktop/CodeInTheResearch/RawData/CPP/RunningforGold/CppPerf/1/sim/sim
name:  XORandDistance
/Users/silalertbanjongngam/Desktop/CodeInTheResearch/RawData/CPP/XORandDistance/CppPerf/1/alpha/alpha


...

/Users/silalertbanjongngam/anaconda3/lib/python3.9/site-packages/researchpy/ttest.py:18: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  groups = group1.append(group2, ignore_index= True)
/Users/silalertbanjongngam/anaconda3/lib/python3.9/site-packages/researchpy/ttest.py:18: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  groups = group1.append(group2, ignore_index= True)
/Users/silalertbanjongngam/anaconda3/lib/python3.9/site-packages/researchpy/ttest.py:18: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  groups = group1.append(group2, ignore_index= True)
/Users/silalertbanjongngam/anaconda3/lib/python3.9/site-packages/researchpy/ttest.py:18: FutureWarning: The series.append method is deprecated and will be removed from pand

/Users/silalertbanjongngam/Desktop/CodeInTheResearch/RawData/PYTHON/Book/PythonPerf/1/sim/sim
name:  RegularBracketSequences
/Users/silalertbanjongngam/Desktop/CodeInTheResearch/RawData/PYTHON/RegularBracketSequences/PythonPerf/1/alpha/alpha
/Users/silalertbanjongngam/Desktop/CodeInTheResearch/RawData/PYTHON/RegularBracketSequences/PythonPerf/1/sim/sim


/Users/silalertbanjongngam/anaconda3/lib/python3.9/site-packages/researchpy/ttest.py:18: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  groups = group1.append(group2, ignore_index= True)
/Users/silalertbanjongngam/anaconda3/lib/python3.9/site-packages/researchpy/ttest.py:18: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  groups = group1.append(group2, ignore_index= True)
/Users/silalertbanjongngam/anaconda3/lib/python3.9/site-packages/researchpy/ttest.py:18: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  groups = group1.append(group2, ignore_index= True)
/Users/silalertbanjongngam/anaconda3/lib/python3.9/site-packages/researchpy/ttest.py:18: FutureWarning: The series.append method is deprecated and will be removed from pand

      name  alMemMean  alMemMed  alMemSD simMemMean simMemMed simMemSD  \
1    1549A    39.2010   39.2090   0.1217    39.4353   39.4336   0.1096   
0    1549C    83.2158   83.2305   0.1259    78.5995   78.6152   0.1287   
2    1551A    39.1544   39.1641   0.1226     39.188   39.1797   0.1253   
3   1551B1    39.1969   39.2109   0.1079    39.1793   39.1836   0.1105   
4    1552A    39.1747   39.1797   0.1220    39.1938   39.1953   0.1168   
6    1552B    53.8854   53.8965   0.1133     53.885   53.9355   0.1344   
5    1553A    39.1811   39.1855   0.1125    39.1672   39.1836   0.1248   
7    1553H     5.8183    5.8180   0.0063         NA        NA       NA   
8    1554A    55.1097   55.0820   0.1666    55.1638   55.1602   0.1779   
9    1554B     0.2195    0.2200   0.0020         NA        NA       NA   
10   1554C    39.1715   39.1992   0.1258    39.1646   39.1738    0.124   
11   1556A    39.4315   39.4395   0.1109    39.1876   39.1953    0.106   
12   1557A    50.1959   50.1953   0.11

/var/folders/l1/dlc_jzs90vj3g01l0jr_866w0000gn/T/ipykernel_51506/409377616.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resMemDF = resMemDF.append({'name':nameMapProblem[i],'alMemMean':tmp[1][0],'alMemMed':tmp[1][1],'alMemSD':tmp[1][2], \
/var/folders/l1/dlc_jzs90vj3g01l0jr_866w0000gn/T/ipykernel_51506/409377616.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resTimeDF = resTimeDF.append({'name':nameMapProblem[i],'alTimeMean':tmp[0][0],'alTimeMed':tmp[0][1],'alTimeSD':tmp[0][2], \
/var/folders/l1/dlc_jzs90vj3g01l0jr_866w0000gn/T/ipykernel_51506/409377616.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resMemDF = resMemDF.append({'name':nameMapProblem[i],'alMemMean':tmp[0][0],'alMemMed':tmp[0][1],'alMemSD':